In [1]:
import os
import sys
import time
import json
import requests

from dotenv import load_dotenv
from functools import reduce
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, row_number, lit
from pyspark.sql.window import Window

# Pemrosesan Data Menggunakan Apache Spark

Percobaan kali ini secara proses hampir sama seperti yang dilakukan pada proses etl sebelumnya, namun satu yang berbeda adalah penggunaan *Apache Spark* untuk pemrosesan data. Yang mana sebelumnya hanya menggunakan dataframe dari pandas. Kekurangan dari penggunaan pandas adalah pada *Eager Execution* dimana ketika butuh proses transformasi yang berulang akan cukup merepotkan. Oleh karena itulah disini spark digunakan, dimana kita bisa menyusun tahapan-tahapan transformasi terlebih dahulu tanpa langsung melakukan transformasi secara eksplisit.

Ada yang berbeda juga kali ini dimana data sebelumnya ada di kota *San Fransisco*, untuk kali ini kami melakukan beberapa penyesuaian yaitu dengan merubah tempat menjadi *Kota Malang*. Adapun untuk sumber data yang kami gunakan masih tetap berasal dari API Open-meteo (https://open-meteo.com/en/docs/historical-weather-api)

In [2]:
# Tambahkan 2 baris ini agar spark mengenali env python
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
# Buat spark session terlebih dahulu
spark = SparkSession.builder \
    .appName("rdv-project") \
    .master("local[*]") \
    .getOrCreate()

# Extract Data

Ekstraksi data yang dilakukan juga akan berbeda dari tugas sebelumnya. Pada tugas sebelumnya hanya menggunakan satu fitur saja yaitu temperature_2m. Namun pada kali ini karena spark memanfaatkan memori untuk pemrosesan data, kami akan mengambil lebih banyak fitur.

List fitur yang akan dipakai : 
- temperature_2m,
- relative_humidity_2m,
- precipitation,
- weather_code,
- wind_speed_10m,
- wind_speed_100m,
- wind_direction_10m,
- wind_direction_100m,
- wind_gusts_10m,
- rain,
- apparent_temperature,
- surface_pressure,
- cloud_cover,
- cloud_cover_low,
- cloud_cover_mid,
- cloud_cover_high,
- et0_fao_evapotranspiration,
- vapour_pressure_deficit,
- pressure_msl,
- dew_point_2m

Bisa dilihat bahwa fitur yang diambil bisa dibilang cukup banyak. Jika hanya untuk dashboard saja yang tujuannya untuk monitoring cuaca saat ini, fitur-fitur seperti informasi keadaan tanah dan lain-lain yang terlalu spesifik mungkin tidak diperlukan. Akan tetapi kebutuhan akan berbeda ketika ada pertimbangan untuk melakukan **prediksi** kode cuaca (*weather code*) dari semua fitur yang ada.

In [4]:
"""
Malang sendiri sebenarnya tidak memiliki stasiun pengukuran dari BMKG
Oleh karena itu latitude dan longitude di bawah ini menunjukkan 
koordinat dari kota malang seluruhnya
"""
# --------------------------- Parameter API -------------------------

# Latitude & Longitude utk Malang
locations = {
    "Klojen":(-7.969421375342755, 112.6285308513895),
    "Blimbing":(-7.945926995201971, 112.64310740385885),
    "Sukun":(-7.987926536974757, 112.6100743302227),
    "Lowokwaru":(-7.9348455775950795, 112.60665573821122),
    "Lawang":(-7.828716329007881, 112.70181673602366),
    "Singosari":(-7.875350536265517, 112.64837233274164),
    "Kepanjen":(-8.11761411293624, 112.57906911889476),
    "Pakis":(-7.958180248694484, 112.71010117082125),
    "Wagir":(-7.980111689103741, 112.49397160866646),
    "Tumpang":(-8.007270828630284, 112.74768804369178)
}

# Tanggal
today = datetime.today() - timedelta(days=3) # dikurang 2 hari karena 2 hari sebelum hari ini selalu kosong/nan
end_date = today.strftime("%Y-%m-%d")

# List Fitur
feature_list = [
    "temperature_2m",
    "relative_humidity_2m",
    "precipitation",
    "weather_code",
    "wind_speed_10m",
    "wind_speed_100m",
    "wind_direction_10m",
    "wind_direction_100m",
    "wind_gusts_10m",
    "rain",
    "apparent_temperature",
    "surface_pressure",
    "cloud_cover",
    "cloud_cover_low",
    "cloud_cover_mid",
    "cloud_cover_high",
    "et0_fao_evapotranspiration",
    "vapour_pressure_deficit",
    "pressure_msl",
    "dew_point_2m"
]
features = ",".join(feature_list)

In [ ]:
final_psdf = None

for district in locations.keys() :
    print(f"Start fetch data for {district}")
    latitude = locations[district][0]
    longitude = locations[district][1]
    for year in range(2017, 2025+1):
        start_date = f"{year}-01-01"
        temp_end_date = f"{year}-12-31"
        
        if year == 2025:
            temp_end_date = end_date
            
        url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={temp_end_date}&hourly={features}&timezone=auto"
        
        success = False
        attempts = 0

        while not success and attempts < 3:
            try:
                print(f"Getting data for {year} from {start_date} to {temp_end_date}")
                res = requests.get(url)

                if res.status_code == 200:
                    data = res.json()['hourly']
                    num_records = len(data['time'])
                    
                    rows = [
                        {key: data[key][i] for key in data}
                        for i in range(num_records)
                    ]
                    
                    psdf = spark.read.json(spark.sparkContext.parallelize(rows))
                    psdf = psdf.withColumn("district", lit(district))
                    psdf = psdf.withColumn("latitude", lit(latitude))
                    psdf = psdf.withColumn("longitude", lit(longitude))

                    if final_psdf is None:
                        final_psdf = psdf
                    else:
                        final_psdf = final_psdf.unionByName(psdf)

                    if "_corrupt_record" in psdf.columns:
                        psdf = psdf.drop("_corrupt_record")
                    
                    success = True
                    print(f"Successfully fetched {num_records} records for {year}.")
                    time.sleep(60)

                elif res.status_code == 429:
                    print(f"Rate limit reached, retrying... (Attempt {attempts + 1}/3)")
                    time.sleep(60)
                    attempts += 1
                elif res.status_code == 443:
                    print(f"Error status code 443, retrying... (Attempt {attempts + 1}/3)")
                    time.sleep(60)
                    attempts += 1
                else:
                    print(f"Failed to get data for {year}. Status code: {res.status_code}")
                    break
            except Exception as e:
                print(f"Error while getting data for {year}. Message: {e}. Retrying...")
                time.sleep(60)
                attempts += 1

        if not success:
            print(f"Failed to retrieve data for {year} after {attempts} attempts.")

    if final_psdf is not None:
        print(f"Total records fetched: {final_psdf.count()}")
    else:
        print("No data was successfully retrieved.")

    print(f"Done fetch data for {district}")

Start fetch data for Klojen
Getting data for 2017 from 2017-01-01 to 2017-12-31
Successfully fetched 8760 records for 2017.
Getting data for 2018 from 2018-01-01 to 2018-12-31
Successfully fetched 8760 records for 2018.
Getting data for 2019 from 2019-01-01 to 2019-12-31
Successfully fetched 8760 records for 2019.
Getting data for 2020 from 2020-01-01 to 2020-12-31
Successfully fetched 8784 records for 2020.
Getting data for 2021 from 2021-01-01 to 2021-12-31
Successfully fetched 8760 records for 2021.
Getting data for 2022 from 2022-01-01 to 2022-12-31
Successfully fetched 8760 records for 2022.
Getting data for 2023 from 2023-01-01 to 2023-12-31
Successfully fetched 8760 records for 2023.
Getting data for 2024 from 2024-01-01 to 2024-12-31
Successfully fetched 8784 records for 2024.
Getting data for 2025 from 2025-01-01 to 2025-05-14
Rate limit reached, retrying... (Attempt 1/3)
Getting data for 2025 from 2025-01-01 to 2025-05-14
Rate limit reached, retrying... (Attempt 2/3)
Getting 

Tanpa menambahkan timedelta(days=2) atau pengurangan 2 hari, akan muncul corrupt record yang mana berisi data yang semuanya adalah nan. Untuk mempermudah pemrosesan, langsung saja data 2 hari tersebut kami kurangi dari end_date

# Transform Data

## Structuring

Tahap structuring sudah dimulai bersamaan dengan ekstrak data dari API open-meteo. Adapun structuring yang dilakukan adalah sebagai berikut :
* this is an example

In [23]:
# Cek tipe data apakah koalas / sql.dataframe
type(final_psdf)

pyspark.sql.dataframe.DataFrame

In [9]:
# df.head(20) alike
final_psdf.show()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
# Cek apakah semua kolom berhasil terambil, jika semua terambil maka jumlahnya adalah 29
print(f"Jumlah Kolom yang terambil : {len(final_psdf.columns)}")
final_psdf.columns

Jumlah Kolom yang terambil : 29


['apparent_temperature',
 'cloud_cover',
 'cloud_cover_high',
 'cloud_cover_low',
 'cloud_cover_mid',
 'dew_point_2m',
 'et0_fao_evapotranspiration',
 'precipitation',
 'pressure_msl',
 'rain',
 'relative_humidity_2m',
 'soil_moisture_0_to_7cm',
 'soil_moisture_100_to_255cm',
 'soil_moisture_28_to_100cm',
 'soil_moisture_7_to_28cm',
 'soil_temperature_0_to_7cm',
 'soil_temperature_100_to_255cm',
 'soil_temperature_28_to_100cm',
 'soil_temperature_7_to_28cm',
 'surface_pressure',
 'temperature_2m',
 'time',
 'vapour_pressure_deficit',
 'weather_code',
 'wind_direction_100m',
 'wind_direction_10m',
 'wind_gusts_10m',
 'wind_speed_100m',
 'wind_speed_10m']

In [25]:
# Cek jumlah baris data
rows = final_psdf.count()
print(rows)

659664


## Enriching

Enriching pada projek ini dilakukan dengan tujuan untuk mendapatkan sebagian data yang tidak ada. Pada open-meteo data yang kita minta tidak bersifat real-time. Semisal mengambil data hingga hari ini, maka 40 data sebelumnya semuanya *missing*. Oleh karena itu kami melakukan enriching pada 40 data yang hilang tersebut pada API lain yaitu `weatherapi.com`

### Cari tahu pada tanggal berapa data missing

In [26]:
# Pengecekan Missing Values
missing_counts = final_psdf.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in final_psdf.columns
])

In [27]:
print("Missing Values Tiap-Tiap Kolom")
missing_counts.show()

Missing Values Tiap-Tiap Kolom
+--------------------+-----------+----------------+---------------+---------------+------------+--------------------------+-------------+------------+----+--------------------+----------------------+--------------------------+-------------------------+-----------------------+-------------------------+-----------------------------+----------------------------+--------------------------+----------------+--------------+----+-----------------------+------------+-------------------+------------------+--------------+---------------+--------------+--------+
|apparent_temperature|cloud_cover|cloud_cover_high|cloud_cover_low|cloud_cover_mid|dew_point_2m|et0_fao_evapotranspiration|precipitation|pressure_msl|rain|relative_humidity_2m|soil_moisture_0_to_7cm|soil_moisture_100_to_255cm|soil_moisture_28_to_100cm|soil_moisture_7_to_28cm|soil_temperature_0_to_7cm|soil_temperature_100_to_255cm|soil_temperature_28_to_100cm|soil_temperature_7_to_28cm|surface_pressure|tempera

In [28]:
# Cek apakah data yang missing itu pada data terbaru
tail_df = final_psdf.orderBy("time", ascending=False).limit(5)
tail_df.show()

+--------------------+-----------+----------------+---------------+---------------+------------+--------------------------+-------------+------------+----+--------------------+----------------------+--------------------------+-------------------------+-----------------------+-------------------------+-----------------------------+----------------------------+--------------------------+----------------+--------------+----------------+-----------------------+------------+-------------------+------------------+--------------+---------------+--------------+---------+
|apparent_temperature|cloud_cover|cloud_cover_high|cloud_cover_low|cloud_cover_mid|dew_point_2m|et0_fao_evapotranspiration|precipitation|pressure_msl|rain|relative_humidity_2m|soil_moisture_0_to_7cm|soil_moisture_100_to_255cm|soil_moisture_28_to_100cm|soil_moisture_7_to_28cm|soil_temperature_0_to_7cm|soil_temperature_100_to_255cm|soil_temperature_28_to_100cm|soil_temperature_7_to_28cm|surface_pressure|temperature_2m|          

Ternyata data paling baru ada di tanggal 2025-05-12 pada jam 07.00 dan nilai yang hilang bukan di akhir timestamp

In [ ]:
# Cek index dari data yang bernilai nan
window_spec = Window.orderBy(lit(1))
df_with_index = final_psdf.withColumn("row_index", row_number().over(window_spec) - 1)

for column in psdf.columns:
    null_condition = df_with_index.filter(col(column).isNull())
    
    null_indices = null_condition.select("row_index")
    print(f"Indeks yang null pada kolom '{column}':")
    null_indices.show(truncate=False)

Indeks yang null pada kolom 'apparent_temperature':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'cloud_cover':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'cloud_cover_high':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'cloud_cover_low':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'cloud_cover_mid':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'dew_point_2m':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'et0_fao_evapotranspiration':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'precipitation':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'pressure_msl':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'rain':
+---------+
|row_index|
+---------+
+---------+

Indeks yang null pada kolom 'relative_humidity

Kita sudah menemukan masalahnya, ternyata data hilang dari index 3152 (berlaku untuk semua kolom). Itu artinya data hilang dari index tanggal 1 Januari 2017 pukul 00.00 hingga 1 Januari 2017 adalah 12 Mei 2017. Setelah kami cek secara langsung ternyata memang dari `weatherapi.com` tidak memiliki data pada rentang itu. Hasil outputnya ketika requests tanggal tersebut hanya tanggal/timestamp saja.

### Impute nilai missing dari `weatherapi.com`

In [ ]:
load_dotenv()
FREEWEATHER_KEY = os.getenv("FREEWEATHER_KEY")

In [92]:
# Parameter untuk melengkapi data hingga tanggal saat ini terlebih dahulu
# Hanya untuk start date, end date masih sama
# start_date = "2025-01-01"
start_date = "2025-02-05"
today = datetime.today()
end_date = today.strftime("%Y-%m-%d")

weatherapi_url = f"http://api.weatherapi.com/v1/history.json?key={FREEWEATHER_KEY}&q={latitude},{longitude}&dt=2025-01-01&end_dt={end_date}"

# Melakukan pengambilan data baru
response = requests.get(weatherapi_url)
if response.status_code == 200:
    recent_data = response.json()
else :
    print(f"Can't fetch data, status code: {response.status_code}")

In [ ]:
# Melakukan pengambilan data yang hilang dari api lain
start_date = "2017-01-01"
end_date = "2017-05-12"

weatherapi_url = f"http://api.weatherapi.com/v1/history.json?key={FREEWEATHER_KEY}&q={latitude},{longitude}&dt={start_date}&end_dt={end_date}"

response = requests.get(weatherapi_url)
if response.status_code == 200:
    missing_data = response.json()
else :
    print(f"Can't fetch data, status code: {response.status_code}")

Can't fetch data, status code: 400


Setelah kami lihat lebih detail ternyata response data dari `weatherapi.com` hanya mencakup sampai tanggal 5 Feburari 2025 saja, sedangkan yang kami butuhkan adalah data 2 hari sebelum tanggal saat ini. Selain itu karena juga ada batasan data terkait *constraint* tahun, dimana data dari tahun 2017 hingga 2022 selalu ada nilai yang hilang. Akhirnya pada notebook sebelumnya data dimulai dari 2012, kami mengubahnya menjadi mulai dari tahun 2023.

## Cleaning

Untuk mengatasi masalah nilai yang hilang pada 2 hari sebelum hari ini, kami memutuskan untuk menggunakan data forecast dari open-meteo.

In [31]:
print(features)

temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,rain,apparent_temperature,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,et0_fao_evapotranspiration,vapour_pressure_deficit,pressure_msl,dew_point_2m


In [ ]:
for district in locations:
    latitude = locations[district][0]
    longitude = locations[district][1]
    
    forecast_url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly={features}&start_date=2025-05-12&end_date={end_date}"

    response = requests.get(forecast_url)
    if response.status_code == 200:
        missing_data = response.json()['hourly']
        rows_oriented_missing = [
            {key: missing_data[key][i] for key in missing_data}
            for i in range(len(missing_data['time']))
        ]
        missing_psdf = spark.read.json(spark.sparkContext.parallelize(rows_oriented_missing))
        missing_psdf = missing_psdf.withColumn("district", lit(district))
        # Gabungkan data yang baru (2 hari sebelum hari ini) dengan data historis
        cleaned_psdf = final_psdf.unionByName(missing_psdf)
        cleaned_psdf = final_psdf.dropDuplicates()
    else :
        print(f"Can't fetch data, status code: {response.status_code}")

In [47]:
cleaned_psdf = cleaned_psdf.orderBy("district", "time")

In [48]:
cleaned_psdf.show()

+--------------------+-----------+----------------+---------------+---------------+------------+--------------------------+-------------+------------+----+--------------------+----------------+--------------+----------------+-----------------------+------------+-------------------+------------------+--------------+---------------+--------------+--------+
|apparent_temperature|cloud_cover|cloud_cover_high|cloud_cover_low|cloud_cover_mid|dew_point_2m|et0_fao_evapotranspiration|precipitation|pressure_msl|rain|relative_humidity_2m|surface_pressure|temperature_2m|            time|vapour_pressure_deficit|weather_code|wind_direction_100m|wind_direction_10m|wind_gusts_10m|wind_speed_100m|wind_speed_10m|district|
+--------------------+-----------+----------------+---------------+---------------+------------+--------------------------+-------------+------------+----+--------------------+----------------+--------------+----------------+-----------------------+------------+-------------------+----

In [49]:
# Cek apakah data yang missing itu pada data terbaru
tail_df = cleaned_psdf.orderBy("time", ascending=False).limit(5)
tail_df.show()

+--------------------+-----------+----------------+---------------+---------------+------------+--------------------------+-------------+------------+----+--------------------+----------------+--------------+----------------+-----------------------+------------+-------------------+------------------+--------------+---------------+--------------+---------+
|apparent_temperature|cloud_cover|cloud_cover_high|cloud_cover_low|cloud_cover_mid|dew_point_2m|et0_fao_evapotranspiration|precipitation|pressure_msl|rain|relative_humidity_2m|surface_pressure|temperature_2m|            time|vapour_pressure_deficit|weather_code|wind_direction_100m|wind_direction_10m|wind_gusts_10m|wind_speed_100m|wind_speed_10m| district|
+--------------------+-----------+----------------+---------------+---------------+------------+--------------------------+-------------+------------+----+--------------------+----------------+--------------+----------------+-----------------------+------------+-------------------+--

## Validating

# Load Data

In [ ]:
def save_to_couchdb(df, couchdb_url, db_name, username=None, password=None):
    # Convert DataFrame ke list of dicts
    records = df.toPandas().to_dict(orient='records')  # Jika data besar, gunakan df.rdd.map

    # Auth
    auth = (username, password) if username and password else None

    # Buat database jika belum ada
    db_url = f"{couchdb_url}/{db_name}"
    response = requests.put(db_url, auth=auth)
    if response.status_code not in [201, 412]:  # 412 = already exists
        raise Exception(f"Gagal membuat database: {response.text}")

    # Kirim data dalam bulk
    bulk_url = f"{db_url}/_bulk_docs"
    payload = {"docs": records}
    headers = {"Content-Type": "application/json"}

    response = requests.post(bulk_url, data=json.dumps(payload), headers=headers, auth=auth)
    if response.status_code != 201:
        raise Exception(f"Gagal menyimpan data: {response.text}")
    else:
        print("Data berhasil disimpan ke CouchDB.")

# Contoh penggunaan
save_to_couchdb(df, couchdb_url="http://localhost:5984", db_name="mydb", username="admin", password="password")


In [5]:
import datetime
datetime.datetime.now().isoformat()

'2025-05-17T10:57:20.922109'